In [1]:
# !pip install xmltodict


In [2]:
# import pandas as pd

In [3]:
# df = pd.read_xml('/content/Restaurants_Train_v2.xml')

In [14]:
# df

t ="abc"
tt = t.split()
tt

['abc']

In [32]:
Term=  ['chef']
a = ['An', 'oasis', 'of', 'refinement', 'Food', 'though', 'somewhat', 'uneven', 'often', 'reaches', 'the', 'pinnacles', 'of', 'new', 'American', 'fine', 'cuisine', '', "chef's", 'passion', 'and', "kitchen's", 'precise', 'execution', 'is', 'most', 'evident', 'in', 'the', 'fish', 'dishes', 'and', 'soups'] 
for k,i in enumerate(a):
    if Term[0] in i:
        print(k)
        print(i)

18
chef's


In [49]:
import xmltodict

# Open and read the XML file
with open('Restaurants_Train_v2.xml', 'r') as f:
    xml_content = f.read()

# Convert the XML to a dictionary
xml_dict = xmltodict.parse(xml_content)

# Access data as a dictionary
# print(xml_dict)

# Access specific elements
# print(xml_dict['root']['child'])


In [50]:
import re

def get_clean_text(text):
    # Step 1: Remove single or double quotes surrounding words
    cleaned_string = re.sub(r"(['\"])(\w+)\1", r"\2", text)

    # Step 2: Remove any remaining special characters except apostrophes
    cleaned_string = re.sub(r"[^A-Za-z0-9\s']", '', cleaned_string)
    return cleaned_string



In [51]:
def get_ner(split_text,term,from_to,ner_label):
  # split_text = text.split()
  split_term = term.split()

  next=0
  last=0
  word_range=[]
  t_word_range=[]
  index_range=[]

  for k,v in enumerate(split_text):
    word_range.append(v)
    t_word_range.append( get_clean_text(v))
    if v[0] not in ["'",'"']:
      index_range.append(next)
    else:
      index_range.append(next+1)
    # print(v[0])
    if k == 0:
      last = len(v)+1
       
    else:
      last=last+len(v)+1
       
    next = last
  t_index = 0
  try:
    t_index = index_range.index(from_to)
    # print(t_index)
  except:
     
    try:
      t_index = t_word_range.index(split_term[0] )
    except:
      for k,i in enumerate(t_word_range):
        if split_term[0] in i:
          t_index = k
       
  end = t_index + len(split_term)
  
  for k,v in enumerate(range(t_index,end)):
    if k==0:
      ner_label[v]= 1
    else:
      ner_label[v]= 2
  # print(ner_label)
  return word_range, ner_label

In [52]:
def get_dataset(xml_dict):
  dataset_aspect = []
  all_lb = set()
  for k,v in enumerate(xml_dict['sentences']['sentence']):
    ds = {}
    # if k == 5:
    #   break
    dict_keys = v.keys()
    ds['text'] = v['text']
    ds['NER_INFERENCE'] ={}
    ds['NER_INFERENCE']['term']=[]
    ds['NER_INFERENCE']['polarity']=[]
    ds['NER_INFERENCE']['from_to']=[]
    ds['NER_INFERENCE']['text']= v['text'].split()
    ds['NER_INFERENCE']['ner_label']= [0]*len(v['text'].split())
    ds['categories']={}
    ds['categories']['category']=[]
    ds['categories']['polarity']=[]
    # print( v['aspectCategories'])
    # print(dict_keys)
    if 'aspectTerms' in dict_keys:
      if type(v['aspectTerms']['aspectTerm'])==dict:
        # tmp_dict={}
        ds['NER_INFERENCE']['term'].append(v['aspectTerms']['aspectTerm']['@term'])
        ds['NER_INFERENCE']['polarity'].append(v['aspectTerms']['aspectTerm']['@polarity'])
        ds['NER_INFERENCE']['from_to'].append((int(v['aspectTerms']['aspectTerm']['@from']),int(v['aspectTerms']['aspectTerm']['@to'])))
        ds['NER_INFERENCE']['text'],ds['NER_INFERENCE']['ner_label']= get_ner(
                                                                                ds['NER_INFERENCE']['text'],
                                                                                v['aspectTerms']['aspectTerm']['@term'],
                                                                                int(v['aspectTerms']['aspectTerm']['@from']),
                                                                                  ds['NER_INFERENCE']['ner_label']
                                                                              )

      else:
        for key,value in enumerate(v['aspectTerms']['aspectTerm']):
          # tmp_dict={}
          ds['NER_INFERENCE']['term'].append(value['@term'])
          ds['NER_INFERENCE']['polarity'].append(value['@polarity'])
          ds['NER_INFERENCE']['from_to'].append((int(value['@from']),int(value['@to'])))
          # try:
          ds['NER_INFERENCE']['text'],ds['NER_INFERENCE']['ner_label']= get_ner(
                                                                                ds['NER_INFERENCE']['text'],
                                                                                value['@term'],
                                                                                int(value['@from']),
                                                                                  ds['NER_INFERENCE']['ner_label']
                                                                              )
          # except Exception as e:
          #   print(e)
          #   print('Text: ',ds['text'],' Term: ',value['@term']," from: ",int(value['@from']))
          #   return "error"

    if 'aspectCategories' in dict_keys :
      # print( v['aspectCategories'])
      if type(v['aspectCategories']['aspectCategory'])==dict:
        # print( v['aspectCategories']['aspectCategory']) #['@category']
        ds['categories']['category'].append(v['aspectCategories']['aspectCategory']['@category'])
        ds['categories']['polarity'].append(v['aspectCategories']['aspectCategory']['@polarity'])
        all_lb.add(v['aspectCategories']['aspectCategory']['@category'])
      else:
        # print(v['aspectCategories']['aspectCategory'])
        for key,value in enumerate(v['aspectCategories']['aspectCategory']):
          # print(value['@category'])
          ds['categories']['category'].append(value['@category'])
          ds['categories']['polarity'].append(value['@polarity'])
          all_lb.add(value['@category'])
      # pass
    dataset_aspect.append(ds)
  return dataset_aspect,all_lb

In [53]:
dataset_apect,all_lb= get_dataset(xml_dict)

In [54]:
all_lb

{'ambience', 'anecdotes/miscellaneous', 'food', 'price', 'service'}

In [55]:
dataset_apect

[{'text': 'But the staff was so horrible to us.',
  'NER_INFERENCE': {'term': ['staff'],
   'polarity': ['negative'],
   'from_to': [(8, 13)],
   'text': ['But', 'the', 'staff', 'was', 'so', 'horrible', 'to', 'us.'],
   'ner_label': [0, 0, 1, 0, 0, 0, 0, 0]},
  'categories': {'category': ['service'], 'polarity': ['negative']}},
 {'text': "To be completely fair, the only redeeming factor was the food, which was above average, but couldn't make up for all the other deficiencies of Teodora.",
  'NER_INFERENCE': {'term': ['food'],
   'polarity': ['positive'],
   'from_to': [(57, 61)],
   'text': ['To',
    'be',
    'completely',
    'fair,',
    'the',
    'only',
    'redeeming',
    'factor',
    'was',
    'the',
    'food,',
    'which',
    'was',
    'above',
    'average,',
    'but',
    "couldn't",
    'make',
    'up',
    'for',
    'all',
    'the',
    'other',
    'deficiencies',
    'of',
    'Teodora.'],
   'ner_label': [0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,


In [46]:
# text = """Don't go alone---even two people isn't 'enough' for the whole experience, with pickles and a selection of meats and seafoods."""
# from nltk.tokenize import TweetTokenizer
# tokenizer = TweetTokenizer()

# tokenizer.tokenize(text)


In [47]:
# text = "Don't go alone---even two people isn't enough for the whole experience, with pickles and a selection of meats and seafoods."
# term="selection of meats and seafoods"
# from_to =(91,122)

# split_text = text.split()
# ner_label = [0]*len(split_text)
# # print(split_text)
# def get_ner(split_text,term,from_to,ner_label):
#   # split_text = text.split()
#   split_term = term.split()

#   next=0
#   last=0
#   word_range=[]
#   index_range=[]
#   for k,v in enumerate(split_text):
#     word_range.append(v)
#     index_range.append(next)
#     if k == 0:
#       last = len(v)+1
#     else:
#       last=last+len(v)+1
#     next = last
#   t_index = index_range.index(from_to[0])
#   end = t_index + len(split_term)
#   for k,v in enumerate(range(t_index,end)):
#     if k==0:
#       ner_label[v]= 1
#     else:
#       ner_label[v]= 2
#   # print(ner_label)
#   return word_range, ner_label
#   # print(split_text)
# word_range, ner_label=get_ner(split_text,term,from_to,ner_label)

# word_range, ner_label